In [2]:
import torch
import numpy as np
import multiprocessing as mp
import graph_alg2

import time

In [3]:
def sinkhorn_knopp(matrix, max_iter=1000, tol=1e-6):
    for _ in range(max_iter):
        matrix /= matrix.sum(axis=1, keepdims=True)  # 行ごとに正規化
        matrix /= matrix.sum(axis=0, keepdims=True)  # 列ごとに正規化
        
        if np.allclose(matrix.sum(axis=1), 1, atol=tol) and np.allclose(matrix.sum(axis=0), 1, atol=tol):
            break
    
    return matrix


def generate_permutation_array(N, num_goods):
    P = np.zeros((N, num_goods))
    for i in range(N): 
        P[i] = np.random.permutation(num_goods)
    
    return P


def sample_ranking(N, num_goods):
        """ 
        Samples ranked lists
        Arguments
            N: Number of samples
            prob: Probability of truncation       
        Returns:
            Ranked List of shape [N, Num_agents]
        """
        P = generate_permutation_array(N, num_goods) + 1
        
        return P / num_goods

    
def generate_batch(batch_size, num_goods):
        """
        Samples a batch of data from training
        Arguments
            batch_size: number of samples
            prob: probability of truncation
        Returns
            P: Agent's preferences, 
                P_{ij}: How much Agent-i prefers to be Good-j
        """
        N = batch_size * num_goods
        P = sample_ranking(N, num_goods).reshape(-1, num_goods, num_goods)
        return P

In [ ]:
num_iterations = 1000  # 繰り返し回数
batch_size = 1
num_goods = 4

for i in range(num_iterations):
    start_time = time.perf_counter()  # 開始時刻の計測
    
    # ランダムな割り当て行列の生成と処理
    randomassignment = np.random.rand(batch_size, num_goods, num_goods)
    RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
    
    # 好み（preferences）の生成
    preferences = generate_batch(batch_size, num_goods)
    
    # numpy配列から PyTorch の tensor に変換
    P = torch.tensor(RandomAssignment, dtype=torch.float32)
    Preferences = torch.tensor(preferences, dtype=torch.float32)
    
    # 入力データの表示
    print(f"Iteration {i+1}")
    print("P: ", P)
    print("Preferences: ", Preferences)
    
    # グラフアルゴリズムによる処理
    ev = graph_alg2.compute_ev(P, Preferences)
    violations, cycles = ev.execute_all_cycles_batch()
    print("Violations:", violations)
    end_time = time.perf_counter()  # 終了時刻の計測
    print(f"Time taken for iteration {i+1}: {end_time - start_time:.6f} sec")
    print("Cycles:", cycles)
    #print("CPU Count:", cpucount)
    
    
    print("-" * 50)

In [3]:
np.random.seed(0)
batch_size = 5
num_goods = 4
randomassignment = np.random.rand(batch_size, 4, 4)
RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
preferences = generate_batch(batch_size, num_goods)
P = torch.tensor(RandomAssignment, dtype=torch.float32)[:4, :, :]
Preferences = torch.tensor(preferences, dtype=torch.float32)[:4, :, :]

ev = graph_alg2.compute_ev(P, Preferences)
violations, cpucount = ev.execute_all_cycles_batch()
print("P: ", P)
print("Preferences: ", Preferences)
print(violations)
print(cpucount)

ValueError: too many values to unpack (expected 2)

In [ ]:
batch_size = 1
num_goods = 4
randomassignment = np.random.rand(batch_size, 4, 4)
RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
preferences = generate_batch(batch_size, num_goods)
P = torch.tensor(RandomAssignment, dtype=torch.float32)
Preferences = torch.tensor(preferences, dtype=torch.float32)
print("P: ", P)
print("Preferences: ", Preferences)

ev = graph_alg2.compute_ev(P, Preferences)
violations, cpucount = ev.execute_all_cycles_batch()
print(violations)
print(cpucount)

P:  tensor([[[0.2569, 0.2329, 0.2575, 0.2527],
         [0.1816, 0.1907, 0.3265, 0.3012],
         [0.3356, 0.3833, 0.0749, 0.2062],
         [0.2258, 0.1931, 0.3412, 0.2399]]])
Preferences:  tensor([[[0.7500, 1.0000, 0.5000, 0.2500],
         [0.7500, 0.2500, 0.5000, 1.0000],
         [0.7500, 0.5000, 1.0000, 0.2500],
         [1.0000, 0.7500, 0.2500, 0.5000]]])
tensor([[2.0717]])
8


In [ ]:
num_iterations = 100  # 繰り返し回数
batch_size = 1
num_goods = 4

for i in range(num_iterations):
    start_time = time.perf_counter()  # 開始時刻の計測
    
    # ランダムな割り当て行列の生成と処理
    randomassignment = np.random.rand(batch_size, num_goods, num_goods)
    RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
    
    # 好み（preferences）の生成
    preferences = generate_batch(batch_size, num_goods)
    
    # numpy配列から PyTorch の tensor に変換
    P = torch.tensor(RandomAssignment, dtype=torch.float32)
    Preferences = torch.tensor(preferences, dtype=torch.float32)
    
    # 入力データの表示
    print(f"Iteration {i+1}")
    print("P: ", P)
    print("Preferences: ", Preferences)
    
    # グラフアルゴリズムによる処理
    ev = graph_alg2.compute_ev(P, Preferences)
    violations, cpucount = ev.execute_all_cycles_batch()
    #print("Violations:", violations)
    #print("CPU Count:", cpucount)
    
    end_time = time.perf_counter()  # 終了時刻の計測
    print(f"Time taken for iteration {i+1}: {end_time - start_time:.6f} sec")
    #print("-" * 3)

Iteration 1
P:  tensor([[[0.1875, 0.0645, 0.6727, 0.0753],
         [0.2534, 0.2596, 0.0024, 0.4847],
         [0.2888, 0.3875, 0.3080, 0.0157],
         [0.2704, 0.2884, 0.0170, 0.4242]]])
Preferences:  tensor([[[0.7500, 1.0000, 0.2500, 0.5000],
         [0.7500, 0.5000, 1.0000, 0.2500],
         [0.2500, 1.0000, 0.5000, 0.7500],
         [0.5000, 0.7500, 0.2500, 1.0000]]])
Time taken for iteration 1: 0.987341 sec
Iteration 2
P:  tensor([[[0.0011, 0.5488, 0.3061, 0.1440],
         [0.1874, 0.1743, 0.3442, 0.2941],
         [0.3569, 0.2418, 0.2290, 0.1723],
         [0.4546, 0.0350, 0.1207, 0.3897]]])
Preferences:  tensor([[[0.2500, 1.0000, 0.7500, 0.5000],
         [0.7500, 0.5000, 0.2500, 1.0000],
         [0.7500, 0.5000, 1.0000, 0.2500],
         [0.7500, 0.2500, 0.5000, 1.0000]]])
Time taken for iteration 2: 0.757951 sec
Iteration 3
P:  tensor([[[0.2694, 0.1471, 0.3485, 0.2349],
         [0.2172, 0.2477, 0.3725, 0.1626],
         [0.3846, 0.1147, 0.0027, 0.4981],
         [0.1288,

In [ ]:
num_iterations = 100  # 繰り返し回数
batch_size = 1
num_goods = 4

for i in range(num_iterations):
    start_time = time.perf_counter()  # 開始時刻の計測
    
    # ランダムな割り当て行列の生成と処理
    randomassignment = np.random.rand(batch_size, num_goods, num_goods)
    RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
    
    # 好み（preferences）の生成
    preferences = generate_batch(batch_size, num_goods)
    
    # numpy配列から PyTorch の tensor に変換
    P = torch.tensor(RandomAssignment, dtype=torch.float32)
    Preferences = torch.tensor(preferences, dtype=torch.float32)
    
    # 入力データの表示
    print(f"Iteration {i+1}")
    print("P: ", P)
    print("Preferences: ", Preferences)
    
    # グラフアルゴリズムによる処理
    ev = graph_alg2.compute_ev(P, Preferences)
    violations, cpucount = ev.execute_all_cycles_batch()
    #print("Violations:", violations)
    #print("CPU Count:", cpucount)
    
    end_time = time.perf_counter()  # 終了時刻の計測
    print(f"Time taken for iteration {i+1}: {end_time - start_time:.6f} sec")
    #print("-" * 3)

Iteration 1
P:  tensor([[[0.2960, 0.4485, 0.2074, 0.0482],
         [0.2878, 0.0536, 0.3058, 0.3527],
         [0.0352, 0.1744, 0.4693, 0.3211],
         [0.3810, 0.3235, 0.0174, 0.2780]]])
Preferences:  tensor([[[0.2500, 1.0000, 0.7500, 0.5000],
         [0.2500, 1.0000, 0.5000, 0.7500],
         [0.5000, 1.0000, 0.7500, 0.2500],
         [0.7500, 1.0000, 0.2500, 0.5000]]])
Time taken for iteration 1: 1.043186 sec
Iteration 2
P:  tensor([[[0.1731, 0.3990, 0.2901, 0.1379],
         [0.2591, 0.0757, 0.3369, 0.3284],
         [0.2281, 0.4508, 0.1697, 0.1514],
         [0.3398, 0.0745, 0.2033, 0.3824]]])
Preferences:  tensor([[[0.7500, 1.0000, 0.2500, 0.5000],
         [0.2500, 0.5000, 1.0000, 0.7500],
         [0.5000, 0.2500, 0.7500, 1.0000],
         [0.7500, 1.0000, 0.2500, 0.5000]]])
Time taken for iteration 2: 0.724410 sec
Iteration 3
P:  tensor([[[0.2234, 0.2669, 0.2863, 0.2234],
         [0.2119, 0.1879, 0.2607, 0.3396],
         [0.2927, 0.4620, 0.1451, 0.1002],
         [0.2721,